In [20]:
# import de bibliotecas necessárias para o treinamento
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Carrega o arquivo CSV para um DataFrame
df = pd.read_csv('./dadosEmpresas.csv')

# Adiciona a coluna 'nomeFantasia' (ou o nome da empresa correspondente) ao DataFrame
if 'nomeFantasia' not in df.columns:
    raise ValueError("A coluna 'nomeFantasia' não está presente no DataFrame.")

# Seleciona as colunas de interesse: endividamento, passivoCirculante, faturamentoBruto
colunas_interesse = ['endividamento', 'passivoCirculante', 'faturamentoBruto']

# Remove valores nulos dessas colunas
df = df.dropna(subset=colunas_interesse)

# Define as colunas de entrada (X) e a coluna-alvo (target - simulação de risco: 0 para baixo risco, 1 para alto risco)
X = df[colunas_interesse]
y = (df['maiorAtraso'] > 30).astype(int)  # Exemplo: empresas com maior atraso > 30 dias são classificadas como risco

# Aplica o SMOTE para balancear as classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Divide os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Inicializa e treina o modelo RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Avalia o modelo
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Adiciona a coluna 'previsao_risco' ao DataFrame original
df['previsao_risco'] = clf.predict(X)

# Seleciona apenas as colunas 'nomeFantasia' e 'previsao_risco' para o arquivo final
resultado_final = df[['nomeFantasia', 'previsao_risco']]

# Salva o resultado final em um arquivo CSV
resultado_final.to_csv('previsao_risco_final.csv', index=False)


              precision    recall  f1-score   support

           0       0.87      0.81      0.84      1540
           1       0.81      0.86      0.83      1407

    accuracy                           0.84      2947
   macro avg       0.84      0.84      0.84      2947
weighted avg       0.84      0.84      0.84      2947

